In [3]:
import torch

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

print('CUDA 사용 가능 여부 :', USE_CUDA)
print('현재 사용 device :', device)
print('CUDA Index :', torch.cuda.current_device())
print('GPU 이름 :', torch.cuda.get_device_name())
print('GPU 개수 :', torch.cuda.device_count())


CUDA 사용 가능 여부 : True
현재 사용 device : cuda:0
CUDA Index : 0
GPU 이름 : A100-SXM4-40GB
GPU 개수 : 1


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import xgboost as xgb
from matplotlib import rc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint,expon,reciprocal
import time
import warnings
warnings.filterwarnings('ignore')

In [4]:
def make_table(weather_info_csv, num):
    #df_electronic = pd.read_csv('/Users/gyujin/Library/CloudStorage/OneDrive-개인/POSTECH 테양열 발전 경진대회/cmpt2022_data1/gens.csv')
    df_electronic = pd.read_csv('gens.csv')
    df_electronic['time'] = df_electronic['time'].str.split('+').str[0]
    df_electronic['time'] = pd.to_datetime(df_electronic['time'])
    df_weather = pd.read_csv(weather_info_csv)
    df_weather = df_weather.rename(columns = {'id' : 'weather_id'})
    df_weather['time'] = df_weather['time'].str.split('+').str[0]
    df_weather['time'] = pd.to_datetime(df_weather['time'])
    df_num = df_electronic[df_electronic['id'] == num]
    df_weather['time'] = df_weather['time'].dt.round(freq = 'H')  
    df_weather.drop_duplicates(['time'], inplace = True)
    df_info = pd.merge(df_weather,df_num, how='outer')
    df_info = df_info.dropna()
    df_info = df_info.drop(['id'], axis = 1)
    return df_info

In [5]:
#df_11 = make_table('/Users/gyujin/Library/CloudStorage/OneDrive-개인/POSTECH 테양열 발전 경진대회/발전소날씨/발전소11_기상정보.csv', 11)
df_11 = make_table('발전소11_기상정보.csv', 11)
df_12 = make_table('발전소12_기상정보.csv', 12)
df_13 = make_table('발전소13_기상정보.csv', 13)
df_14 = make_table('발전소14_기상정보.csv', 14)
df_17 = make_table('발전소17_기상정보.csv', 17)

In [6]:
df_total = pd.concat([df_11,df_12,df_13,df_14,df_17])
df_total

,weather_id,time,temperature,humidity,dew_point,wind_dir,wind_spd,uv_idx,visibility,cloudiness,ceiling,pressure,precip_1h,amount
1,12.0,2020-10-01 01:00:00,15.4,90.0,13.9,68.0,3.0,0.0,12.9,51.0,9144.0,1012.0,0.0,0.0
2,12.0,2020-10-01 02:00:00,15.1,97.0,14.6,45.0,3.4,0.0,14.5,100.0,9144.0,1012.0,0.0,0.0
3,12.0,2020-10-01 03:00:00,14.9,98.0,14.6,135.0,5.0,0.0,11.3,100.0,9144.0,1013.0,0.0,0.0
4,12.0,2020-10-01 04:00:00,14.7,93.0,13.6,68.0,4.2,0.0,11.3,91.0,9144.0,1013.0,0.0,0.0
5,12.0,2020-10-01 05:00:00,14.5,99.0,14.3,113.0,5.3,0.0,11.3,99.0,9144.0,1012.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17744,18.0,2022-10-23 20:00:00,11.6,63.0,4.8,338.0,5.8,0.0,16.1,6.0,12192.0,1020.0,0.0,0.0
17745,18.0,2022-10-23 21:00:00,12.5,49.0,2.1,338.0,4.3,0.0,16.1,0.0,12192.0,1020.0,0.0,0.0
17746,18.0,2022-10-23 22:00:00,11.6,52.0,2.2,338.0,4.3,0.0,20.9,5.0,12192.0,1021.0,0.0,0.0
17747,18.0,2022-10-23 23:00:00,10.2,56.0,1.9,338.0,4.3,0.0,16.1,5.0,12192.0,1021.0,0.0,0.0


In [8]:
x = df_total.iloc[:, 2 : 13]
y = df_total.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 1) 

In [9]:
param_distribs = {
      'n_estimators': randint(low = 1, high = 200),
        'max_depth' : randint(low = 1, high = 200),
      'max_features': randint(low = 1, high = 8),
  }
forest_reg = RandomForestRegressor(random_state = 42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X_train, y_train)

In [ ]:
rnd_search.best_params_ #randomforest regressor

In [ ]:
param_distribs = {
 'n_estimators': [ 500, 1000 ],
 "learning_rate"    : [ 0.02, 0.05 ] ,
 "max_depth"        : [ 3, 5 ],
 "min_child_weight" : [ 2, 4 ],
 "gamma"            : [ 0.05 ],
 'subsample' : [ 0.6, 0.8 ]}

xgb_reg=xgb.XGBRegressor()
rnd_search = RandomizedSearchCV(xgb_reg, param_distributions=param_distribs, n_iter=10, cv=5, scoring='neg_mean_squared_error',random_state=42)
rnd_search.fit(X_train, y_train)

In [ ]:
rnd_search.best_params_ #XGB regressor

In [ ]:
"""param_distribs = {
    'kernel': ['linear','rbf','poly'],
    'C' :randint(low=0.01,high=10),
    'gamma' : randint(low=0.01,high=10),
}
svm_reg =SVR()
#svm_reg.get_params().keys()
rnd_search = RandomizedSearchCV(svm_reg,param_distributions=param_distribs,n_iter=10,cv=5,scoring="neg_mean_squared_error")
rnd_search.fit(X_train,y_train)"""

In [ ]:
"""rnd_search.best_params_ #svm regressor"""

In [ ]:
param_distribs = {'alpha': randint(low=0.00001, high=10)}
lasso_reg=Lasso()
rnd_search = RandomizedSearchCV(lasso_reg, param_distributions=param_distribs, n_iter=10, cv=5, scoring='neg_mean_squared_error')
rnd_search.fit(X_train, y_train)

In [ ]:
rnd_search.best_params_ #lasso 

In [ ]:
param_distribs = {'alpha': randint(low=0.001, high=100)}
Ridge_reg=Ridge()
rnd_search = RandomizedSearchCV(Ridge_reg, param_distributions=param_distribs, n_iter=10, cv=5, scoring='neg_mean_squared_error')
rnd_search.fit(X_train, y_train)

In [ ]:
rnd_search.best_params_ #ridge

In [ ]:
#('SVR',SVR(kernel='poly',gamma=0.1)),
#('Lasso',Lasso(alpha=0)),

In [19]:
base_models = [
    ('XGB', xgb.XGBRegressor(subsample=0.6,n_estimators=1000,min_child_weight=4,max_depth=5,learning_rate=0.05,gamma=0.05)),
    ('Random Forest',RandomForestRegressor(random_state =1, max_features = 5, n_estimators = 103, max_depth = 189)),
    ('MLP Regression',MLPRegressor(random_state=1, max_iter=500)),
    ('Ridge',Ridge(alpha=16))
    ]
stacked = StackingRegressor(
    estimators = base_models,
    final_estimator = LinearRegression(), cv = 5)

In [20]:
for name, model in base_models:
    start_time = time.time()
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    end_time = time.time()
    
    r2 = model.score(X_test, y_test)
    rmse = mean_squared_error(y_test, prediction, squared = False)
    
    print("-------{}-------".format(name))
    print("Coefficient of determination: {}".format(r2))
    print("Root Mean Squared Error: {}".format(rmse))
    print("Computation Time: {}".format(end_time - start_time))
    print("----------------------------------\n")
start_time = time.time()
stacked.fit(X_train, y_train)    
stacked_prediction = stacked.predict(X_test)
end_time = time.time()
stacked_r2 = stacked.score(X_test, y_test)
stacked_rmse = mean_squared_error(y_test, stacked_prediction, squared = False)
print("-------Stacked Ensemble-------")
print("Coefficient of determination: {}".format(stacked_r2))
print("Root Mean Squared Error: {}".format(stacked_rmse))
print("Computation Time: {}".format(end_time - start_time))
print("----------------------------------")

[08:19:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
-------XGB-------
Coefficient of determination: 0.893795099569257
Root Mean Squared Error: 0.32885376116532017
Computation Time: 99.86375641822815
----------------------------------

-------Random Forest-------
Coefficient of determination: 0.8930841553288367
Root Mean Squared Error: 0.3299526121934088
Computation Time: 16.35642671585083
----------------------------------

-------MLP Regression-------
Coefficient of determination: 0.8784421630939435
Root Mean Squared Error: 0.35182120919947246
Computation Time: 25.44287919998169
----------------------------------

-------Ridge-------
Coefficient of determination: 0.752468735632141
Root Mean Squared Error: 0.5020483745955029
Computation Time: 0.020652294158935547
----------------------------------

[08:22:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:sq

In [ ]:
#기본모델(xgb,randomforest,mlp)
#모델 4개 ,svr 포함 Coefficient of determination: 0.9019551471168702
#모델 3개 svr 제외 0.9019532732882437
#모델 4개 lasso 추가 0.9019542875695038
#모델 5개 lasso,rigde 추가 0.9019550196646348
#모델 4개 ridge 추가 0.9019540144584349


1. 11월 9일 데이터프레임 불러오기
2. 불러온 데이터프레임 X넣기(발전량 제외해서)
3. 학습시킨 스테킹 모델 돌리기
4. 예측한 데이터 데이터프레임으로 뽑아서 실제 발전량과 비교해보기

In [21]:
import joblib
joblib.dump(model, './stacked_11~14.pkl')

['./stacked_11~14.pkl']

In [22]:
loaded_model = joblib.load('./stacked_11~14.pkl')


In [85]:
x_1109 = pd.read_csv("train1109.csv")
x_1109.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           17 non-null     int64  
 1   time         17 non-null     object 
 2   temperature  17 non-null     float64
 3   humidity     17 non-null     int64  
 4   dew_point    17 non-null     float64
 5   wind_dir     17 non-null     int64  
 6   wind_spd     17 non-null     float64
 7   uv_idx       17 non-null     int64  
 8   visibility   17 non-null     float64
 9   cloudiness   17 non-null     int64  
 10  ceiling      17 non-null     int64  
 11  pressure     17 non-null     float64
 12  precip_1h    17 non-null     int64  
 13  amount       17 non-null     int64  
dtypes: float64(5), int64(8), object(1)
memory usage: 2.0+ KB


In [88]:
x_1109['time'] = x_1109['time'].str.split('+').str[0]
x_1109['time'] = pd.to_datetime(x_1109['time'])

In [89]:
x_1109['time']=x_1109["time"].values.astype("float")

In [92]:
x_1109.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           17 non-null     int64  
 1   time         17 non-null     float64
 2   temperature  17 non-null     float64
 3   humidity     17 non-null     int64  
 4   dew_point    17 non-null     float64
 5   wind_dir     17 non-null     int64  
 6   wind_spd     17 non-null     float64
 7   uv_idx       17 non-null     int64  
 8   visibility   17 non-null     float64
 9   cloudiness   17 non-null     int64  
 10  ceiling      17 non-null     int64  
 11  pressure     17 non-null     float64
 12  precip_1h    17 non-null     int64  
 13  amount       17 non-null     int64  
dtypes: float64(6), int64(8)
memory usage: 2.0 KB


In [95]:
train_1109 = x_1109.iloc[:, 2 : 13]

In [99]:
Train_1109 = scaler.fit_transform(train_1109)
Train_1109

array([[-0.22148805, -0.05263462, -0.48659133,  0.65955452, -0.75818166,
        -0.73854895,  1.04274033, -1.54805975,  1.63057664,  0.23342489,
         0.        ],
       [-0.37947254,  0.24562821, -0.42041491,  0.83914074, -0.90300288,
        -0.73854895, -0.4803076 , -1.7635207 , -0.22233429, -2.10082401,
         0.        ],
       [-0.53745702, -0.27633174, -1.54541407,  0.65955452,  0.32797746,
        -0.73854895, -0.81876269,  1.1913723 , -0.59285569,  0.23342489,
         0.        ],
       [-1.22205645,  1.28954811, -0.94982628,  0.83914074, -1.19264531,
        -0.73854895,  0.36583014, -1.05557758, -0.22233429, -0.23342489,
         0.        ],
       [-1.06407197,  0.46932533, -1.67776691, -1.79999594, -0.68577106,
        -0.73854895, -0.98799024, -1.08635772, -1.23207348,  0.23342489,
         0.        ],
       [-1.09040271,  1.28954811, -0.61894417,  0.65955452, -0.03407558,
        -0.73854895, -0.98799024,  1.1913723 , -1.23207348,  0.23342489,
         0.   

In [97]:
stacked_prediction = stacked.predict(Train_1109)
stacked_prediction

array([-0.58561633, -0.65626718, -0.57886263, -0.66546291, -0.69413067,
       -0.62758706, -0.62885954, -0.67009587, -0.66680251, -0.53777858,
        0.38919965,  1.17690558,  0.90852518,  1.60885888,  1.52128247,
        1.34150398,  0.70840252])

In [98]:
pd.DataFrame(stacked_prediction)

,0
0,-0.585616
1,-0.656267
2,-0.578863
3,-0.665463
4,-0.694131
5,-0.627587
6,-0.628860
7,-0.670096
8,-0.666803
9,-0.537779
